In [1]:
"""
Since constraints and rules are essentially the same thing, they should be unified into the same type of object. 
Things to consider in this process:

    - The Dwarf Planet constraint is similar to, but not the same as a Band Rule. A Band Rule does not require the
        objects to be at the two ends of the band; they can be anywhere within. It's not an "exact band" but a 
        maximum requirement
    - The Comet constraint is not modeled by any of the rules. A "restricted positions" type rules should be made 
        to accommodate this
    - This will allow the generate_rule/eliminate_sectors methods to take in a set of constraints and compare it to
        themselves to determine if generating a certain rule makes sense
    - It might make sense to unify SelfRule and RelationRule in the process
        - The three types of self rules are equivalent to the three types of relation rules
        - However, the comet-type rule would not be equivalent to any relation rule that I can think of
    - Constraints always had qualifiers of "none" or "every", not "at least one", which affected the fill_board 
        method since it could make fewer assumptions.
"""

'\nSince constraints and rules are essentially the same thing, they should be unified into the same type of object. \nThings to consider in this process:\n\n    - The Dwarf Planet constraint is similar to, but not the same as a Band Rule. A Band Rule does not require the\n        objects to be at the two ends of the band; they can be anywhere within. It\'s not an "exact band" but a \n        maximum requirement\n    - The Comet constraint is not modeled by any of the rules. A "restricted positions" type rules should be made \n        to accommodate this\n    - This will allow the generate_rule/eliminate_sectors methods to take in a set of constraints and compare it to\n        themselves to determine if generating a certain rule makes sense\n    - It might make sense to unify SelfRule and RelationRule in the process\n        - The three types of self rules are equivalent to the three types of relation rules\n        - However, the comet-type rule would not be equivalent to any relation

In [2]:
import itertools
import random
import json
from enum import Enum
from abc import *

In [3]:
from server.utilities import permutations_multi, fill_no_touch, fill_no_within

In [4]:
from server.board import *
from server.board_type import *

In [5]:
from server.game import RuleQualifier

In [6]:
class Rule(ABC):
    @abstractmethod
    def is_satisfied(self, board):
        """
        Checks whether a board meets this constraint. Returns true if constraint is met.
        """
        pass
    
    @abstractmethod
    def is_immediately_limiting(self):
        """
        Returns true if this constraint limits the positions the space object can be in
        even when no objects are on the board.
        """
        pass

    @abstractmethod
    def disallowed_sectors(self):
        """
        If the constraint is immediately limited, returns the list of sector that the 
        space object relevant to the constraint is not allowed to be in. Otherwise,
        returns [].
        """
        return []
    
    @abstractmethod
    def fill_board(self, board, num_objects):
        """
        Given a board and the number of each space object that must appear on it,
        fills the board in all ways possible given this constraint.
        
        board: A partially filled Board
        num_objects: A dictionary mapping space objects to the number of times they
            are supposed to appear in the board.
            
        Returns: a list of boards that contain the same objects as the original board,
            plus filling in objects to meet this constraint. Returns all possible
            such boards.
        """
        pass
    
    @abstractmethod
    def affects(self):
        """
        Returns a list of space objects that are affected by this rule. Note that this is
        not the same as all of the space objects named in the constraint - e.g.
            "All gas clouds are next to an empty sector" does not affect empty sectors,
            since the other empty sectors may be wherever they like
        """
        pass
    
    @abstractmethod
    def completes(self):
        """
        Returns a list of all space object that this constraint can "complete" - that is, 
        when generating all possible boards using fill_board, this constraint will add in 
        every object of that type.
        """
        pass
    
    @abstractmethod
    def adds(self):
        """
        Returns a list of all space objects that can be added during fill_board
        """
        pass
    
    @abstractmethod
    def space_objects(self):
        """
        Returns all the space objects involved in this rule
        """
        pass
    
    @classmethod
    @abstractmethod
    def generate_rule(cls, board, constraints, *space_objects):
        """
        Generates a rule of this type for a particular board and space objects to relate 
        to each other. Returns None if no such rule exists, or if such a rule would be 
        redundant with the given constraints.
        """
        pass
    
    @abstractmethod
    def code(self):
        """
        Return compressed string code that represents this rule
        """
        pass
    
    @classmethod
    @abstractmethod
    def parse(cls, s):
        """
        Parse a compressed string representation s into a rule of this type
        """
        pass
    
    @abstractmethod
    def text(self, board):
        """
        Returns a readable text version of this rule given an input Board board.
        """
        pass
    
    @abstractmethod
    def to_json(self, board):
        """
        Create a json representation of this rule, given a particular board. The json
        representation should have the following fields:
            - ruleType: type of the rule
            - If the rule is a self-rule:
                - spaceObject: the space object in the rule
            - If the rule is a relation-rule:
                - spaceObject1: the space object in the rule
                - spaceObject2: the space object which spaceObject1 is related to in the rule
            - categoryName: the name of the category the rule is in (based on types of objects)
            - text: a readable text representation of the rule
            - other rule-specific fields   
        """
        pass

In [7]:
class RelationRule(Rule):
    def space_object1(self):
        return self.space_object1
    
    def space_object2(self):
        return self.space_object2
    
    def space_objects(self):
        return [self.space_object1(), self.space_object2()]
    
    @classmethod
    @abstractmethod
    def eliminate_sectors(cls, board, data, space_object1, space_object2):
        """
        Create a rule that will eliminate possible positions of space_object1, where 
        space_object1 and data.elimination_object are ambiguous on survey/target
        
        Only the sectors in data.need_eliminated are ambiguous, and the sectors in 
        data.already_eliminated have been eliminated by other rules. To be viable,
        this rule must eliminate at least data.minimum sectors, and if possible 
        should eliminate data.goal sectors.
        """
        pass

In [8]:
class SelfRule(Rule):
    def space_object(self):
        return self.space_object
    
    def space_objects(self):
        return [self.space_object()]

In [62]:
class WithinRule(RelationRule):
    """
    A rule stating that two objects are or aren't within a certain number of sectors from each other
    """
    def __init__(self, space_object1, space_object2, qualifier, num_sectors):
        self.space_object1 = space_object1
        self.space_object2 = space_object2
        self.qualifier = qualifier
        self.num_sectors = num_sectors
        
    def __repr__(self):
        return "<" + self.qualifier.name + " " + repr(self.space_object1) + " within " + str(self.num_sectors) + \
                " sectors of " + repr(self.space_object2) + ">"
    
    def __str__(self):
        return str(self.qualifier) + " " + self.space_object1.name() + " is within " + str(self.num_sectors) + \
                " sectors of " + self.space_object2.one() + " " + self.space_object2.name() + "."
    
    def text(self, board):
        num_object1 = board.num_objects()[self.space_object1]
        num_object2 = board.num_objects()[self.space_object2]
        return self.qualifier.for_object(self.space_object1, num_object1) + " within " + \
                str(self.num_sectors) + " sectors of " + self.space_object2.any_of(num_object2) + "."
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            if self.qualifier == other.qualifier:
                if self.qualifier is RuleQualifier.NONE or self.qualifier is RuleQualifier.AT_LEAST_ONE:
                    # Mutual rules - i.e. space objects can be swapped
                    return self.space_object1 == other.space_object1 and self.space_object2 == other.space_object2 or \
                            self.space_object1 == other.space_object2 and self.space_object2 == other.space_object1
                else:
                    # One way rules - i.e. space objects cannot be swapped
                    return self.space_object1 == other.space_object1 and self.space_object2 == other.space_object2
            else:
                return False
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)
        
    
    def _is_satisfied_none(self, board):
        prev = None
        countdown = 0
        is_valid = True
        for i in range(-self.num_sectors, len(board)):
            obj = board[i]
            if obj is self.space_object1 or obj is self.space_object2:
                if countdown != 0 and obj != prev:
                    is_valid = False
                    break
                prev = obj
                countdown = self.num_sectors
            else:
                countdown = max(0, countdown - 1)
                
        return is_valid
    
    def _num_within(self, board):
        prev = None, None
        countdown = 0
        within_indices = set()
        for i in range(-self.num_sectors, len(board)):
            if board[i] is self.space_object1:
                if countdown > 0 and prev[0] is self.space_object2:
                    within_indices.add(i % len(board))
                prev = board[i], i
                countdown = self.num_sectors
            elif board[i] is self.space_object2:
                if countdown > 0 and prev[1] is not None:
                    within_indices.add(prev[1] % len(board))
                prev = board[i], i
                countdown = self.num_sectors
            else:
                countdown = max(0, countdown - 1)
        return len(within_indices)
    
    def is_satisfied(self, board):
        if self.qualifier is RuleQualifier.NONE:
            return self._is_satisfied_none(board)
        else:
            num_within = self._num_within(board)
            if self.qualifier is RuleQualifier.AT_LEAST_ONE:
                return num_within > 0
            else:
                return num_within == board.num_objects()[self.space_object1]
    
    def is_immediately_limiting(self):
        return False

    def disallowed_sectors(self):
        return []
    
    def _fill_board_none(self, board, num_objects):
        if not self.is_satisfied(board):
            # There are already some within this range, cannot meet rule
            return []
                
        num_obj1 = num_objects[self.space_object1]
        num_obj2 = num_objects[self.space_object2]
        
        if self.space_object1 in board.num_objects():
            num_obj1 -= board.num_objects()[self.space_object1]
        
        if self.space_object2 in board.num_objects():
            num_obj2 -= board.num_objects()[self.space_object2]
                
        return [Board(b) for b in fill_no_within({self.space_object1: num_obj1, self.space_object2: num_obj2}, \
                                                 board, self.num_sectors)]
   
    def _fill_board_every(self, board, num_objects, num_objects_left, start_i=0):
        # num_objects: how many should be on the board starting from start_i
        # num_objects_left: how many still need to be placed
        num_obj1 = num_objects[self.space_object1]
        num_obj2 = num_objects[self.space_object2]
        
        num_obj1_left = num_objects_left[self.space_object1]
        num_obj2_left = num_objects_left[self.space_object2]
                
        new_num_objects = num_objects.copy()
        new_num_objects_left = num_objects_left.copy()

        if num_obj1 == 0:
            return [ board ]
        
        new_boards = []
        for i in range(start_i, len(board)):
            # Try to add obj1 
            # keep track of countdown until obj2 is needed
            # might have troouble with wraparound effects
            # try every obj2 position in radius
            # similar rules for adjacency: if between obj1s, must be on the right
            obj = board[i]
            is_obj1 = obj is self.space_object1 
            # Attempt to fill each position withna object1 is possible
            if obj is None or is_obj1:
                options = 0
                if self.space_object2 in board[i-self.num_sectors:i+self.num_sectors+1]:
                    # If there is already an obj2 in range, fill with obj1 and proceed
                    options += 1
                    board_copy = board.copy()
                    board_copy[i] = self.space_object1
                    new_num_objects[self.space_object1] = num_obj1 - 1
                    new_num_objects[self.space_object2] = num_obj2
                    new_num_objects_left[self.space_object1] = num_obj1_left - (not is_obj1)
                    new_num_objects_left[self.space_object2] = num_obj2
                    new_boards.extend(self._fill_board_every(board_copy, new_num_objects, new_num_objects_left, i+1))
                elif num_obj2_left > 0:
                    # Otherwise there must be obj2 left to use
                    for j in range(i-self.num_sectors, i):
                        if board[j] is None:
                            if not any(k >= 0 and board[k] is self.space_object1 for k in range(j-self.num_sectors, j)):
                                options += 1
                                board_copy = board.copy() 
                                board_copy[i] = self.space_object1
                                board_copy[j] = self.space_object2
                                new_num_objects[self.space_object1] = num_obj1 - 1
                                new_num_objects[self.space_object2] = num_obj2 - 1
                                new_num_objects_left[self.space_object1] = num_obj1_left - (not is_obj1)
                                new_num_objects_left[self.space_object2] = num_obj2_left - 1
                                new_boards.extend(self._fill_board_every(board_copy, new_num_objects, new_num_objects_left, i+1))
                    
                    max_sector = min(i+self.num_sectors+1, len(board)+(i-self.num_sectors))
                    for j in range(i+1, max_sector):
                        if board[j] is None:
                            if not any(k < len(board) and board[k] is self.space_object1 for k in range(j+1, j+self.num_sectors+1)):
                                options += 1
                                board_copy = board.copy() 
                                board_copy[i] = self.space_object1
                                board_copy[j] = self.space_object2
                                new_num_objects[self.space_object1] = num_obj1 - 1
                                new_num_objects[self.space_object2] = num_obj2 - 1
                                new_num_objects_left[self.space_object1] = num_obj1_left - (not is_obj1)
                                new_num_objects_left[self.space_object2] = num_obj2_left - 1
                                new_boards.extend(self._fill_board_every(board_copy, new_num_objects, new_num_objects_left, i+1))
        
        return new_boards
 
    def fill_board(self, board, num_objects):
        if self.qualifier is RuleQualifier.NONE:
            return self._fill_board_none(board, num_objects)
        elif self.qualifier is RuleQualifier.AT_LEAST_ONE:
            # Not yet supported
            return None
        else:
            num_objects_left = num_objects.copy()
            for obj in board:
                if obj is self.space_object1 or obj is self.space_object2:
                    num_objects_left[obj] -= 1
                
            return self._fill_board_every(board, num_objects, num_objects_left)
            
    def affects(self):
        if self.qualifier is RuleQualifier.NONE:
            return [ self.space_object1, self.space_object2 ]
        elif self.qualifier is RuleQualifier.AT_LEAST_ONE:
            return []
        else:
            return [ self.space_object1 ]
    
    def completes(self):
        if self.qualifier is RuleQualifier.NONE:
            return [ self.space_object1, self.space_object2 ]
        elif self.qualifier is RuleQualifier.AT_LEAST_ONE:
            return []
        else:
            return [ self.space_object1 ]
    
    def adds(self):
        return [ self.space_object1, self.space_object2 ]
    
    @staticmethod
    def _circle_dist(i, j, size):
        """
        The distance between i and j on a circle of size size - i.e. modular distance
        """
        dist = abs(i - j)
        return min(dist, size - dist)
    
    @staticmethod
    def _max_min_sectors_away(space_object1, space_object2, board):
        """
        Calculate the minimum and maximum number of sectors any space_object1 is from space_object2
        """
        board_size = len(board)
        obj1_positions = [i for i, obj in enumerate(board) if obj is space_object1]
        obj2_positions = [i for i, obj in enumerate(board) if obj is space_object2]
        
        maximum_sectors = 0
        minimum_sectors = board_size
        for i in obj1_positions:
            # How far away this obj1 is from the nearest obj2
            sectors_away = min(WithinRule._circle_dist(i, j, board_size) for j in obj2_positions)
            if sectors_away > maximum_sectors:
                maximum_sectors = sectors_away
            if sectors_away < minimum_sectors:
                minimum_sectors = sectors_away
        
        return minimum_sectors, maximum_sectors
    
    @classmethod
    def generate_rule(cls, space_object1, space_object2, board):
        num_object1 = board.num_objects()[space_object1]
        num_object2 = board.num_objects()[space_object2]
        
        # There must be more of object 2 (or at least the same amount) than object 1
        if num_object1 > num_object2:
            return None
        
        max_n = int(len(board)/3 - 1)
        
        min_sectors, max_sectors = cls._max_min_sectors_away(space_object1, space_object2, board)
        
        options = []
        
        if min_sectors > 2:
            # Create a random number of sectors that no object1 is within object2
            # This must be < min_sectors, because object1 is within that many sectors of object2.
            num_not_within = random.randrange(2, min_sectors)
            options.append((RuleQualifier.NONE, num_not_within))
        
        if max_sectors <= max_n:
            # Create a random number of sectors that every object1 is within object2
            # This must be at least max_sectors, to cover every possible object1
            num_within = random.randrange(max(2, max_sectors), max_n+1)
            options.append((RuleQualifier.EVERY, num_within))
            
        if len(options) == 0:
            return None
        
        # Choose a random rule
        qualifier, num_sectors = random.choice(options)
        return WithinRule(space_object1, space_object2, qualifier, num_sectors)
    
    @classmethod
    def eliminate_sectors(cls, space_object1, space_object2, data, board):
        """
        Create a rule that will eliminate possible positions of space_object1, where 
        space_object1 and data.elimination_object are ambiguous on survey/target
        
        Only the sectors in data.need_eliminated are ambiguous, and the sectors in 
        data.already_eliminated have been eliminated by other rules. To be viable,
        this rule must eliminate at least data.minimum sectors, and if possible 
        should eliminate data.goal sectors.
        """
        max_n = int(len(board)/3 - 1)
        board_size = len(board)
        obj1_positions = [i for i, obj in enumerate(board) if obj is space_object1]
        el_positions = data.need_eliminated
        obj2_positions = [i for i, obj in enumerate(board) if obj is space_object2]
        
        # Object 2 appears same as object 1
        if data.elimination_object is space_object2:
            obj2_positions += obj1_positions
        
        # Get the maximum and minimum positions that object 1 is away from an "object 2"
        max_obj1 = 0
        min_obj1 = board_size
        for i in obj1_positions:
            sectors_away = min(WithinRule._circle_dist(i, j, board_size) for j in obj2_positions if j != i)
            if sectors_away > max_obj1:
                max_obj1 = sectors_away
            if sectors_away < min_obj1:
                min_obj1 = sectors_away
        
        # Get how many sectors each eliminated object is from an "object 2"
        # Organize them into an array, with each index containing the indices for elimination objects that are that
        # number of sectors from an "object 2"
        max_el = 0
        min_el = board_size
        el_sectors_away = set()
        el_sectors_away = [[]]
        for i in el_positions:
            sectors_away = min(WithinRule._circle_dist(i, j, board_size) for j in obj2_positions if j != i)
            if sectors_away > len(el_sectors_away) - 1:
                el_sectors_away.extend([[] for j in range(sectors_away-len(el_sectors_away)+1)])
            el_sectors_away[sectors_away].append(i)
            if sectors_away > max_el:
                max_el = sectors_away
            if sectors_away < min_el:
                min_el = sectors_away
        
        el_sectors_away = el_sectors_away[:max_n+1]
        
        options = []
        for sectors_away, matching_el_indices in enumerate(el_sectors_away):
            if sectors_away < 2:
                continue
            if sectors_away >= min_obj1:
                # An "every obj1 within n sectors rule" would eliminate the el_objs further than that
                eliminated = set(i for idx_list in el_sectors_away[sectors_away+1:] for i in idx_list)
                if len(eliminated) > 0:
                    eliminated -= data.already_eliminated
                    options.append((sectors_away, eliminated, RuleQualifier.EVERY))
            if sectors_away < max_obj1:
                # A "no obj1 within n sectors rule" would eliminate the el_objs that are within that range
                eliminated = set(i for idx_list in el_sectors_away[:sectors_away+1] for i in idx_list)
                if len(eliminated) > 0:
                    eliminated -= data.already_eliminated
                    options.append((sectors_away, eliminated, RuleQualifier.NONE))

        if len(options) == 0:
            return None, None
                
        max_num_eliminated = max(len(eliminated) for sectors, eliminated, qualifier in options)
        
        # Only consider rules eliminating the goal value if it at least one does
        if max_num_eliminated >= data.goal:
            options = [option for option in options if len(option[1]) >= data.goal]
           
        # In any case, rules must eliminate the minimum number of sectors
        options = [option for option in options if len(option[1]) >= data.minimum]

        if len(options) == 0:
            return None, None
        
        # Generate a random rule from the choices
        rand_rule_opts = random.choice(options)
        
        num_object1 = board.num_objects()[space_object1]
        num_object2 = board.num_objects()[space_object2]
        rand_rule = WithinRule(space_object1, space_object2, rand_rule_opts[2], rand_rule_opts[0])
        
        return rand_rule_opts[1], rand_rule

    def code(self):
        return "W" + str(self.space_object1) + str(self.space_object2) + self.qualifier.code() + str(self.num_sectors)
    
    @classmethod
    def parse(cls, s):
        space_object1 = SpaceObject.parse(s[1])
        space_object2 = SpaceObject.parse(s[2])
        qualifier = RuleQualifier.parse(s[3])
        num_sectors = int(s[4])
        return cls(space_object1, space_object2, qualifier, num_sectors)
    
    def to_json(self, board):
        return {
            "ruleType": "WITHIN",
            "spaceObject1": self.space_object1.to_json(),
            "spaceObject2": self.space_object2.to_json(),
            "numSectors": self.num_sectors,
            "qualifier": self.qualifier.to_json(),
            "categoryName": self.category_name(),
            "text": self.text(board)
        }

In [63]:
w = WithinRule(SpaceObject.Asteroid, SpaceObject.Comet, RuleQualifier.EVERY, 2)

In [74]:
b = Board([None, None, SpaceObject.Comet, None, None, None])

In [75]:
w.fill_board(b, {SpaceObject.Asteroid: 2, SpaceObject.Comet: 2})

[<Board AAC--->,
 <Board A-CA-->,
 <Board A-C-A->,
 <Board A-CC-A>,
 <Board A-C-CA>,
 <Board ACC--A>,
 <Board -ACA-->,
 <Board -AC-A->,
 <Board -AC-CA>,
 <Board CAC--A>,
 <Board --CAA->,
 <Board C-CA-A>,
 <Board -CCA-A>,
 <Board --CCAA>,
 <Board C-C-AA>,
 <Board -CC-AA>]

In [13]:
self[ii] for ii in range(*i.indices(len(self))

SyntaxError: invalid syntax (<ipython-input-13-ef17adbc7159>, line 1)

In [ ]:
slice(1-2,1+2,1).start

In [ ]:
None or 1

In [ ]:
help(slice)